# EventNavigator – Multi-Agent Event & Location Concierge

## 🟦 Problem

In college fests, conferences, hackathons, and campus events, attendees constantly struggle to find:
- “What events are happening today?”
- “Where is Hall A / Room 204?”
- “Where is that AI summit I asked about earlier?”
- “What time does this workshop start?”

Event schedules are:
- hard to search
- non-conversational
- not connected to venue/location data
- not context-aware (can’t remember previous queries)

This wastes time for attendees and overwhelms volunteers/event coordinators.

---

## 🟩 Solution

**EventNavigator** is a **multi-agent AI concierge** that:

- Understands natural language event/location queries  
- Fetches events by date or ID  
- Finds venue details with **fuzzy search** (even if users type partial/wrong names)  
- Maintains **multi-turn conversation memory** so users can ask:
  - “Where is that event?”
  - “Tell me more about the second one.”
  - “What’s the location of the AI workshop?”  
- Connects events ↔ locations seamlessly  

Instead of manually reading schedules/maps, attendees get instant answers.

---

## 🟦 Why Agents?

Agents allow:
### 1. **Separation of Concerns**
- **Event Agent** handles *event logic only*.  
- **Location Agent** handles *venue logic only*.  
- **Coordinator Agent** decides which sub-agent to use.

### 2. **Modular Architecture**
You can easily plug in:
- actual campus map APIs  
- Google Calendar  
- dynamic event backends  

without rewriting the whole system.

### 3. **Contextual Multi-turn Conversations**
Agents + memory allow:
- references like “that event”
- follow-up queries
- deep context understanding

---

## 🟧 Architecture Overview
```
                           ┌───────────────────────────────┐
                           │          EventAgent           │
                           │    (Top-level LlmAgent Router)│
                           └───────────────┬───────────────┘
                                           │
        ┌──────────────────────────────────┼──────────────────────────────────┐
        │                                  │                                  │
        ▼                                  ▼                                  ▼

 ┌──────────────────────┐         ┌────────────────────────┐           (future...)
 │     eventPlanner     │         │       getLocation      │
 │        Agent         │         │         Agent          │
 └──────────┬───────────┘         └─────────────┬──────────┘
            │                                     │
            ▼                                     ▼

     Event Tools:                           Location Tools:
     • get_event_details                     • get_location_by_id
     • get_event_by_id                       • get_location_by_venue_name (fuzzy)
     • get_all_events
     • get_today_date

```
---
## Key Concepts from the Course Used

1. **Multi-Agent System**
   - `eventPlanner` (Agent) – handles event-related tool calls.
   - `getLocation` (Agent) – handles location/venue tool calls.
   - `eventAgent` (`LlmAgent`) – top-level coordinator that decides which agent/tool to use.

2. **Tools**
   - Event tools:
     - `get_event_details(target_date)`
     - `get_event_by_id(event_id)`
     - `get_all_events()`
     - `get_today_date()`
   - Location tools:
     - `get_location_by_id(location_id)`
     - `get_location_by_venue_name(venue_name, fuzzy=True)`
   - Wrapped as `FunctionTool`s and used inside the agents.

3. **Sessions & Memory**
   - Uses `InMemorySessionService` to store conversation state per `session_id`.
   - `run_session()` reuses the same `session_id` so follow-up queries like
     "Where is that event?" work using previous context.

4. **Context Engineering**

    Instructions force structured JSON outputs
    Agents follow deterministic tool-calling rules

---
### Key Features:
- **Modular multi-agent system**
- **Custom FunctionTools**
- **AgentTools calling sub-agents**
- **Session-based memory (InMemorySessionService)**
- **Gemini 2.5 Pro + Flash-Lite models**

---



## 🟫 How to Run

### 1. Install dependencies


In [ ]:

!pip install --upgrade google-adk


# 2. Add your API key


In [ ]:

!export GOOGLE_API_KEY="your-key"



# 3. Run notebook in order:

> Imports

> Data definitions (EVENTS, LOCATIONS)

> Tools

> Agents

> Runner + SessionService

> Demo cells

In [ ]:
from typing import Any, Dict, List, Optional

from google.adk.agents import Agent, LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.sessions import InMemorySessionService
from google.adk.tools import AgentTool, FunctionTool
from google.genai import types

print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


In [ ]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")



✅ Helper functions defined.


In [ ]:
import os
from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )


✅ Gemini API key setup complete.


In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [ ]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"


# Step 1: Create the LLM Agent


# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner


print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


#TOOLS

In [ ]:
from datetime import datetime, date
from typing import List, Dict, Union, Optional

EVENTS = [
    {
        "event_id": "e_01",
        "event_name": "Tech Innovations Summit 2025",
        "description": "A full-day conference about AI, robotics, and emerging technologies.",
        "url": "https://example.com/tech-summit",
        "keywords": ["AI", "tech", "conference", "future"],
        "location_id": "l_01",
        "venue": "Hall A - City Convention Center",
        "start": "2025-05-15T09:00:00"
    },
    {
        "event_id": "e_02",
        "event_name": "Digital Marketing Bootcamp",
        "description": "A hands-on analytics and SEO bootcamp for beginners and professionals.",
        "url": "https://example.com/marketing-bootcamp",
        "keywords": ["marketing", "SEO", "workshop"],
        "location_id": "l_02",
        "venue": "Room 204 - TechPark",
        "start": "2025-05-15T14:00:00"
    },
    {
        "event_id": "e_03",
        "event_name": "Startup Pitch Night",
        "description": "Entrepreneurs pitch their ideas to investors with Q&A and mentoring.",
        "url": "https://example.com/pitch-night",
        "keywords": ["startup", "investment", "pitch"],
        "location_id": "l_03",
        "venue": "Auditorium - Alpha Hub",
        "start": "2025-05-16T18:00:00"
    }
]
# -------------------------------------------------------------------


def _parse_date(d: str) -> date:
    """Internal helper to parse dates in common formats."""
    try:
        return datetime.fromisoformat(d).date()
    except ValueError:
        return datetime.strptime(d, "%d-%b-%Y").date()


# ===================================================================
# 1. GET EVENTS BY DATE
# ===================================================================
def get_event_details(target_date: Union[str, date]) -> List[Dict]:
    """
    Retrieves event information occurring on a specific date.

    Args:
        target_date:
            The target date to search for events.
            Accepts:
                - A string in 'YYYY-MM-DD' format.
                - A string in 'DD-MMM-YYYY' format (e.g., '15-May-2025').
                - A datetime.date object.

    Returns:
        A dictionary of a list of dictionaries, where each dictionary contains details of an
        event happening on the specified date.

        Expected keys include:
        - 'status' : If the retrieval was successful(status = success) or failure (status = error)
        - 'event_id': The unique identifier of the event.
        - 'event_name': The name of the event.
        - 'description': A paragraph describing the event.
        - 'url': A link for more information.
        - 'keywords': A list of relevant keywords.
        - 'location_id': The ID representing the venue.
        - 'venue': The name or room where the event takes place.

    Example success return value:
    {"status" : "success" , "output" :
    [
        {
            'event_id': 'e_01',
            'event_name': 'Tech Innovations Summit 2025',
            'description': 'A full-day conference about AI, robotics, and emerging technologies.',
            'url': 'https://example.com/tech-summit',
            'keywords': ['AI', 'tech', 'conference', 'future'],
            'location_id': 'l_01',
            'venue': 'Hall A - City Convention Center'
        }
    ]
    }

    Example error return values:
    {"status": "error", "output": "Invalid date format. Expected YYYY-MM-DD or DD-MMM-YYYY."}

    {"status": "error", "output": "No events found for the given date."}
    """
    if isinstance(target_date, str):
        target_date = _parse_date(target_date)

    results = []

    for ev in EVENTS:
        ev_date = datetime.fromisoformat(ev["start"]).date()
        if ev_date == target_date:
            results.append({
                "event_id": ev["event_id"],
                "event_name": ev["event_name"],
                "description": ev["description"],
                "url": ev["url"],
                "keywords": ev["keywords"],
                "location_id": ev["location_id"],
                "venue": ev["venue"],
                "start": ev["start"]
            })

    return results


# ===================================================================
# 2. GET EVENT BY EVENT-ID
# ===================================================================
def get_event_by_id(event_id: str) -> Optional[Dict]:
    """
    Retrieves full details of a specific event using its unique event ID.

    Args:
        event_id:
            The unique identifier assigned to the event (e.g., 'e_02').

    Returns:
        A dictionary of a dictionary containing details of the event if found,
        or None if no event matches the given ID.

        Expected keys include:

        - 'status' : If the retrieval was successful(status = success) or failure (status = error)
        - 'event_id': The unique event identifier.
        - 'event_name': The name of the event.
        - 'description': A paragraph describing the event.
        - 'url': A link for additional event details.
        - 'keywords': A list of descriptive keywords.
        - 'location_id': The location identifier.
        - 'venue': The room or place where the event occurs.

    Example success return value:
    {"status" : "success" , "output" :
    {
        'event_id': 'e_02',
        'event_name': 'Digital Marketing Bootcamp',
        'description': 'A hands-on analytics and SEO bootcamp.',
        'url': 'https://example.com/marketing-bootcamp',
        'keywords': ['marketing', 'SEO', 'workshop'],
        'location_id': 'l_02',
        'venue': 'Room 204 - TechPark'
    }
    }

    Example error return values:
    {"status": "error", "output": "event_id cannot be empty."}

    {"status": "error", "output": "No event found with ID '<provided_id>'."}
    """
    for ev in EVENTS:
        if ev["event_id"] == event_id:
            return {
                "event_id": ev["event_id"],
                "event_name": ev["event_name"],
                "description": ev["description"],
                "url": ev["url"],
                "keywords": ev["keywords"],
                "location_id": ev["location_id"],
                "venue": ev["venue"],
                "start": ev["start"]
            }
    return None


# ===================================================================
# 3. GET ALL EVENTS (UTILITY)
# ===================================================================
def get_all_events() -> List[Dict]:
    """
    Retrieves information for all available events in the system.

    Args:
        None

    Returns:
        A dictionary of a list of dictionaries. Each dictionary contains full details of an event.

    Expected keys include:
    - 'status' : If the retrieval was successful(status = success) or failure (status = error)
    - 'event_id' : The unique identifier of the event.
    - 'event_name' : The name of the event.
    - 'description' : A paragraph describing the event.
    - 'url' : A link for more information.
    - 'keywords' : A list of relevant keywords.
    - 'location_id' : The ID representing the venue.
    - 'venue' : The name or room where the event takes place.
    - 'start' : The start time and date of the event.

    Example success return value:
    {"status" : "success" , "output" :
    [
        {
            'event_id': 'e_01',
            'event_name': 'Tech Innovations Summit 2025',
            'description': '...',
            'url': 'https://example.com/tech-summit',
            'keywords': ['AI', 'tech', ...],
            'location_id': 'l_01',
            'venue': 'Hall A - City Convention Center',
            'start' : '2025-05-16T18:00:00'
        },
        ...
    ]
    }

    Example error return value:
    {"status": "error", "output": "No event data available."}
    """
    events_list = []

    for ev in EVENTS:
        events_list.append({
            "event_id": ev["event_id"],
            "event_name": ev["event_name"],
            "description": ev["description"],
            "url": ev["url"],
            "keywords": ev["keywords"],
            "location_id": ev["location_id"],
            "venue": ev["venue"]
        })

    return events_list


def get_event_by_event_name(
    event_name: str,
    fuzzy: bool = True,
    top_n: int = 3,
    threshold: float = 0.6
) -> List[Dict]:
    """
    Retrieves events whose names match the given event name (fuzzy or exact).

    Args:
        event_name:
            The event name to search for. Can be partial, misspelled,
            or out of order.
        fuzzy:
            If True, apply fuzzy matching using token-set and sequence ratio.
        top_n:
            Maximum number of results to return.
        threshold:
            Minimum similarity score to include event in results.

    Returns:
        A dictionary of a list of dictionaries:
        {"status" : "success" , "output" :
        [
            {
                "event": { ...event details... },
                "score": 0.92
            }
        ]
        }

    Example error return values:
    {"status": "error", "output": "Event name cannot be empty."}

    {"status": "error", "output": "No matching events found for the given event name."}
    """

    if not event_name or not event_name.strip():
        return []

    query_norm = _normalize(event_name)
    results = []

    for ev in EVENTS:
        name_norm = _normalize(ev["event_name"])

        # No fuzzy → only exact or substring match
        if not fuzzy:
            if query_norm == name_norm or query_norm in name_norm:
                results.append((ev, 1.0))
            continue

        # Fuzzy score
        score = _combined_score(query_norm, name_norm)

        # Bonus: if user typed fewer words
        query_tokens = set(query_norm.split())
        name_tokens = set(name_norm.split())
        if query_tokens and query_tokens.issubset(name_tokens):
            score = max(score, 0.85)

        if score >= threshold:
            results.append((ev, score))

    # Sort by highest score
    results.sort(key=lambda x: x[1], reverse=True)

    # Format output
    return [
        {
            "event": {
                "event_id": ev["event_id"],
                "event_name": ev["event_name"],
                "description": ev["description"],
                "url": ev["url"],
                "keywords": ev["keywords"],
                "location_id": ev["location_id"],
                "venue": ev["venue"],
                "start": ev["start"]
            },
            "score": round(score, 3)
        }
        for ev, score in results[:top_n]
    ]


def get_today_date() -> dict:

    """
    Retrieves the current date in 'YYYY-MM-DD' format.

    Example success return value:
    {"status": "success", "output": "2025-05-15"}

    Example error return value:
    {"status": "error", "output": "Failed to retrieve today's date."}
    """
    today = str(date.today())
    return {
        "status": "success",
        "output": today}

print("Event tool defined!")


Event tool defined!


In [ ]:
eventPlanner = Agent(
    name="eventPlanner_FetchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''
You are the Event Planner Fetch Agent.

Your responsibilities:
1. Receive a date, an event ID or an event name from the user or nothing(use get_all_events to extract all event data).
2. Determine whether the user is requesting:
   - All events on a specific date, or
   - Details of a specific event by event ID or event name.
   - Asking about all the events.
3. Call the appropriate tool:
   - get_event_details(date)
   - get_event_by_id(event_id)
   - get_event_by_event_name(event_name)
   - get_all_events()
4. Return the tool result as json, exactly provided below — never fabricate event data.
Example :
 {status : "success", output : [{ "event_id": "e_01",
        "event_name": "Tech Innovations Summit 2025",
        "description": "A full-day conference about AI, robotics, and emerging technologies.",
        "url": "https://example.com/tech-summit",
        "keywords": ["AI", "tech", "conference", "future"],
        "location_id": "l_01",
        "venue": "Hall A - City Convention Center",
        "start": "2025-05-15T09:00:00"},{
        "event_id": "e_02",
        "event_name": "Digital Marketing Bootcamp",
        "description": "A hands-on analytics and SEO bootcamp for beginners and professionals.",
        "url": "https://example.com/marketing-bootcamp",
        "keywords": ["marketing", "SEO", "workshop"],
        "location_id": "l_02",
        "venue": "Room 204 - TechPark",
        "start": "2025-05-15T14:00:00"
    },
    ...
    ]}
5. If no event is found, return {status : "error" , output : {provide short description}}
6. If the user mentions "today" , to get today's date, use get_today_date()

Rules:
- When receiving date, make sure it is in YYYY-MM-DD format, otherwise convert the date in YYYY-MM-DD format.
- Do not guess or invent event information.
- Do not perform computations that the tools should handle.
- If the user input is unclear, ask for clarification.
- Always prioritize using the provided tools for retrieving data.
- Always return data in json form.


    '''
  ,
    tools=[get_event_details,get_event_by_id,get_all_events,get_today_date,get_event_by_event_name],
    output_key="event",

)

get_location_tool


In [ ]:

from typing import Dict, List, Optional, Tuple
from difflib import SequenceMatcher
import re

# -------------------------
# SAMPLE LOCATION DATA
# -------------------------
# Fields: location_id, venue, room_name, room_number, floor_number, block_number
_LOCATIONS = [
    {
        "location_id": "l_01",
        "venue": "Hall A - City Convention Center",
        "room_name": "Main Hall",
        "room_number": "A1",
        "floor_number": 1,
        "block_number": "North"
    },
    {
        "location_id": "l_02",
        "venue": "City Convention Centre - Hall B",  # British spelling + ordering variant
        "room_name": "Breakout Room",
        "room_number": "B12",
        "floor_number": 2,
        "block_number": "East"
    },
    {
        "location_id": "l_03",
        "venue": "Alpha Hub",
        "room_name": "Auditorium",
        "room_number": "AU",
        "floor_number": 3,
        "block_number": "Central"
    },
    {
        "location_id": "l_04",
        "venue": "TechPark Campus - Room 204",
        "room_name": "Lab 2",
        "room_number": "204",
        "floor_number": 2,
        "block_number": "South"
    },
    {
        "location_id": "l_05",
        "venue": "Fountain Square (Meet-up Point)",
        "room_name": "Outdoor",
        "room_number": "",
        "floor_number": 0,
        "block_number": ""
    },
]


# -------------------------
# Normalization & Scoring
# -------------------------
def _normalize(text: str) -> str:
    """
    Lowercase, remove punctuation, normalize British/American spelling variants and whitespace.
    """
    if not text:
        return ""
    t = text.lower()
    # normalize common variants
    replacements = {
        r"\bcentre\b": "center",
        r"\bcentre\b": "center",
        r"\bconvention centre\b": "convention center",
        r"\bconvention centre\b": "convention center",
        r"\b–\b": " ",
        r"-": " ",
        r"&": "and",
        r"\s+": " ",
        r"[()\,\.]": ""
    }
    for pattern, repl in replacements.items():
        t = re.sub(pattern, repl, t)
    t = t.strip()
    return t


def _token_set_score(a: str, b: str) -> float:
    """
    Simple token-set similarity: intersection / union of token sets.
    Returns value in [0,1].
    """
    a_tokens = set(a.split())
    b_tokens = set(b.split())
    if not a_tokens or not b_tokens:
        return 0.0
    inter = a_tokens.intersection(b_tokens)
    union = a_tokens.union(b_tokens)
    return len(inter) / len(union)


def _seq_ratio(a: str, b: str) -> float:
    """SequenceMatcher ratio, value in [0,1]."""
    return SequenceMatcher(None, a, b).ratio()


def _combined_score(a: str, b: str) -> float:
    """
    Combine token-set score and sequence ratio into a single confidence score.
    We take the max of both to favor exact token overlap OR high character similarity.
    """
    a_n = _normalize(a)
    b_n = _normalize(b)
    return max(_token_set_score(a_n, b_n), _seq_ratio(a_n, b_n))


# -------------------------
# Public APIs
# -------------------------
def get_location_by_id(location_id: str) -> Optional[Dict]:
    """
    Retrieves location details using the unique location ID.

    Args:
        location_id: The unique identifier for the location (e.g., 'l_02').

    Returns:
        A dictionary containing location details or None if not found.
        Expected keys include:
            'location_id': unique id
            'venue': full venue name as stored
            'room_name': name of the room
            'room_number': the room number string
            'floor_number': integer floor number
            'block_number': identifier (string)

    Example:
        get_location_by_id('l_02') -> {
            'location_id': 'l_02',
            'venue': 'City Convention Centre - Hall B',
            'room_name': 'Breakout Room',
            'room_number': 'B12',
            'floor_number': 2,
            'block_number': 'East'
        }
    """
    for loc in _LOCATIONS:
        if loc.get("location_id") == location_id:
            return dict(loc)  # return a shallow copy
    return {"status": "error", "output": f"No location found for id '{location_id}'."}


def get_location_by_venue_name(
    venue_name: str,
    fuzzy: bool = True,
    top_n: int = 3,
    threshold: float = 0.6
) -> List[Dict]:
    """
    Finds locations by venue name. Supports exact and fuzzy matching.

    Args:
        venue_name:
            The venue name (user input). Can be partial, slightly misspelled,
            or differ in ordering (e.g., user: 'city convention center',
            DB: 'Hall A - City Convention Center').
        fuzzy:
            If True, will perform fuzzy matching and return ranked results.
            If False, perform normalized exact-and-substring matching only.
        top_n:
            Maximum number of results to return (ranked by confidence score).
        threshold:
            Minimum confidence score (0.0-1.0) for including a fuzzy match.
            Typical safe values: 0.6 (lenient), 0.75 (strict).

    Returns:
        A list of dictionaries representing matching locations ordered by
        descending confidence. Each dictionary contains:
            - 'location' : the location dict (location_id, venue, room_name, ...)
            - 'score' : confidence score between 0 and 1

    Example:
        get_location_by_venue_name('city convention center') ->
        [
            {
                'location': {
                    'location_id': 'l_01',
                    'venue': 'Hall A - City Convention Center',
                    ...
                },
                'score': 0.92
            },
            ...
        ]
    """
    if not venue_name or not venue_name.strip():
        return []

    query_norm = _normalize(venue_name)

    candidates: List[Tuple[Dict, float]] = []

    for loc in _LOCATIONS:
        candidate_venue = loc.get("venue", "")
        cand_norm = _normalize(candidate_venue)

        # Fast exact/substring check if fuzzy disabled
        if not fuzzy:
            if query_norm == cand_norm or query_norm in cand_norm or cand_norm in query_norm:
                candidates.append((loc, 1.0))
            continue

        # Compute combined fuzzy score
        score = _combined_score(query_norm, cand_norm)
        # Also boost if query tokens are subset of candidate tokens (user typed short phrase)
        query_tokens = set(query_norm.split())
        cand_tokens = set(cand_norm.split())
        if query_tokens and query_tokens.issubset(cand_tokens):
            # subset match likely very relevant — bump score up toward 1
            score = max(score, 0.85)

        if score >= threshold:
            candidates.append((loc, score))

    # Sort by score descending and return top_n with 'location' + 'score' keys
    candidates.sort(key=lambda x: x[1], reverse=True)
    result = [{"location": dict(loc), "score": round(score, 3)} for loc, score in candidates[:top_n]]
    return result



In [ ]:
# get_location_agent.py
# Assumes: get_location_by_id and get_location_by_venue_name are imported/defined above
# Also assumes retry_config exists and Agent, Gemini are imported like in your example.

getLocation = Agent(
    name="getLocation_FetchAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""
You are the GetLocation Fetch Agent.

Your responsibilities:
1. Receive input from the user which will be one of:
   - A location_id (e.g., "l_02")
   - A venue name or partial venue name (e.g., "city convention center" or "Hall A - City Convention Center")

2. Determine whether the user provided a location_id or a venue name:
   - If the input matches the pattern of an ID (e.g., starts with "l_" or is prefixed with 'id:'), call get_location_by_id(location_id).
   - Otherwise, treat the input as a venue name and call get_location_by_venue_name(venue_name, fuzzy=True, top_n=3, threshold=0.6).

3. Use the returned value from the tools directly and wrap it in the exact response schema below. Do NOT fabricate or modify data.

4. Return behaviour and schema:
   - For get_location_by_id: if found, return:
     { "status": "success",
       "output": {
         "location_id": "l_02",
         "venue": "City Convention Centre - Hall B",
         "room_name": "Breakout Room",
         "room_number": "B12",
         "floor_number": 2,
         "block_number": "East"
       }
     }
     If not found:
     { "status": "error", "output": "No location found for id 'l_02'." }

   - For get_location_by_venue_name: return ranked results (list) with confidence score:
     { "status": "success",
       "output": [
         {
           "location": {
             "location_id": "l_01",
             "venue": "Hall A - City Convention Center",
             "room_name": "Main Hall",
             "room_number": "A1",
             "floor_number": 1,
             "block_number": "North"
           },
           "score": 0.92
         },
         {
           "location": { ... },
           "score": 0.78
         }
       ]
     }
     If no match above threshold:
     { "status": "error", "output": "No matching venues found for 'city convention center'." }

Rules:
- Always use the provided tools to fetch data. Do not invent, extrapolate, or change values.
- If the user input is ambiguous (e.g., matches both an ID and a probable venue name), prefer treating it as an ID if it matches the ID pattern exactly; otherwise treat it as a venue name.
- For fuzzy venue-name matches:
    - Return up to top_n results (default 3) ordered by descending score.
    - If top result has score >= 0.9, you may return it as the single best match in the output list.
    - If top results have scores between 0.6 and 0.9, return them all and let the caller/UI disambiguate.
- If the user asks for clarification or confirms a choice, wait for that input and then re-run the appropriate tool.
- Always return a JSON-like object with keys exactly "status" and "output" as shown.
- Never return None
""",
    tools=[get_location_by_id, get_location_by_venue_name],
    output_key="location"
)


In [ ]:
def get_fest_details() -> dict:
#web-scrapping from the college site.

  return {"status": "SUCCESS","output" : """
  Exphonia 2025 is a vibrant two-day college fest packed with innovation, creativity, and entrepreneurial energy, hosted by Nova Institute of Technology. The celebration opens with the Tech Innovations Summit 2025, where students and industry experts explore the future of AI, robotics, and emerging tech. The excitement continues with a hands-on Digital Marketing Bootcamp, giving participants practical skills in analytics and SEO through interactive sessions. The fest reaches its peak with the highly anticipated Startup Pitch Night, where young innovators present bold ideas to a panel of investors and mentors. Combining tech, business, and creativity, Exphonia 2025 brings together passionate minds for learning, collaboration, and unforgettable campus vibes.""" }

In [ ]:
eventAgent = LlmAgent(
    name="EventAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction='''
You are the EventAgent — the top-level coordinator for event,location and fest queries.
If you want today's date, you can call get_today_date() and procced with query.

Exphonia 2025 is a two-day tech and innovation fest by Nova Institute of Technology, featuring the Tech Innovations Summit, a practical Digital Marketing Bootcamp, and an exciting Startup Pitch Night where students pitch ideas to investors.
If the user asks more about it, use get_fest_deails().
IMPORTANT : Never return None
Your responsibilities:
1. Understand the user’s request and decide whether they need:
   - Event details for a specific date.
   - Event details for a specific event ID.
   - Event details for a specific event name.
   - Location details for an event’s venue.
   - A venue lookup using fuzzy matching.

2. Delegate correctly:
   - For event-related queries, call the eventPlanner agent using AgentTool(eventPlanner).
   - For location or venue-related queries, call the getLocation agent using AgentTool(getLocation).

3. NEVER fabricate event information, dates, venues, descriptions, or location metadata.
   Always rely completely on the outputs returned by the sub-agents and use that response, answer the query in human-understandable form.

IMPORTANT : Never return None.


4. When using tools:
   - Pass exactly the user’s input.
   - Do not modify, guess, or reformat the parameters.
   - Return the tool result exactly as produced.

5. If the user input is unclear:
   - Ask a clarification question instead of calling a tool.

6. Always follow this output structure:
   {
     "status": "success" or "error",
     "output": <tool_output>
   }

7. Examples of when to call each tool:
   - “Events on 15 May 2025” → eventPlanner
   - "Tell me about event Startup Pitch Night" → eventPlanner
   - “Show event e_02” → eventPlanner
   - “Where is Hall A?” → getLocation
   - “Find venue l_03” → getLocation
   - “Location of the event on 15 May?” → First call eventPlanner → use its venue → call getLocation

8. Cross-tool reasoning:
   If the user first asks for an event and then asks:
   “Where is that event happening?”
   — you should use the previously returned event data to fetch the related venue from getLocation.

9. Safety rules:
   - Do not infer missing event details.
   - Do not merge results yourself — always rely on tools.
   - If no data exists, return an error response.
   - Tell your thinking in the chat.

10. If you want today's date, you can call get_today_date() and procced with query.
11. If the user asks about the exphonia , call get_fest_details().
Your main job is to orchestrate the eventPlanner and getLocation agents correctly without ever inventing data.
    ''',   # <-- THIS COMMA WAS MISSING
    tools=[AgentTool(eventPlanner), AgentTool(getLocation),get_today_date,get_fest_details],
    output_key="event",
)


In [ ]:

runner = Runner(agent=eventAgent, app_name=APP_NAME, session_service=session_service)

## 🟩 Demo Commands (copy these into demo cells)
> 📌 Event search

In [ ]:
await run_session(
    runner,
    [
        "What events are happening on 2025-05-15?"
    ],
    "demo-session-1"
)



 ### Session: demo-session-1

User > What events are happening on 2025-05-15?


gemini-2.5-flash-lite >  On May 15, 2025, the following events are scheduled:

*   **Tech Innovations Summit 2025**: A full-day conference about AI, robotics, and emerging technologies, taking place at **Hall A - City Convention Center** starting at **9:00 AM**.
*   **Digital Marketing Bootcamp**: A hands-on analytics and SEO bootcamp for beginners and professionals, held in **Room 204 - TechPark** starting at **2:00 PM**.


> 📌 Follow-up (tests memory)

In [ ]:
await run_session(
    runner,
    [
        "What events are happening on 2025-05-15?",
        "Tell me more about the second event.",
        "Where is that event located?"
    ],
    "demo-session-2"
)



 ### Session: demo-session-2

User > What events are happening on 2025-05-15?
gemini-2.5-flash-lite >  OK. On 2025-05-15, the following events are scheduled:

*   **Tech Innovations Summit 2025**: A full-day conference on AI, robotics, and emerging technologies at Hall A, City Convention Center.
*   **Digital Marketing Bootcamp**: A hands-on workshop on analytics and SEO for all skill levels at Room 204, TechPark.

User > Tell me more about the second event.
gemini-2.5-flash-lite >  I found the following information about the Digital Marketing Bootcamp:

*   **ID**: e_02
*   **Name**: Digital Marketing Bootcamp
*   **Description**: A hands-on analytics and SEO bootcamp for beginners and professionals.
*   **URL**: [https://example.com/marketing-bootcamp](https://example.com/marketing-bootcamp)
*   **Keywords**: marketing, SEO, workshop
*   **Location ID**: l_02
*   **Venue**: Room 204 - TechPark
*   **Start Time**: 2025-05-15T14:00:00

User > Where is that event located?
gemini-2.5-fl

> 📌 Location search with fuzzy match

In [ ]:
await run_session(
    runner,
    [
        "Where is hall b in the city convention cnter?"
    ],
    "demo-session-3"
)



 ### Session: demo-session-3

User > Where is hall b in the city convention cnter?
gemini-2.5-flash-lite >  The City Convention Centre - Hall B is a Breakout Room, room number B12, located on the 2nd floor in the East block.


## 🗨️ Chat With the Agent (Interactive Mode)

You can chat with the agent just like talking to a friend.  
The loop will keep running until you type **"exit"**, **"quit"**, or **"bye"**.

Run the cell below to start the infinite chat session:


In [ ]:
while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Exiting…")
        break

    await run_session(
        runner,
        [user_input],
        "stateful-agentic-session4",
    )



 ### Session: stateful-agentic-session4

User > yoo
gemini-2.5-flash-lite >  Hello! How can I help you today? I can provide details about events, locations, and fests.
You: exit
Exiting…


In [ ]:
await run_session(
    runner,
    [
       ""  # This time, the agent should remember!
    ],
    "stateful-agentic-session4",
)


 ### Session: stateful-agentic-session4

User > which floor is this event Startup Pitch Night happening?
gemini-2.5-flash-lite >  The Startup Pitch Night is happening on the 3rd floor.
